In [18]:


# {year} {series ID} {Race ID}
year = 2025
series_id = 1
race_id = "ABC456"

In [24]:
import warnings
import requests
import pandas as pd

# endpoint for race list
#https://cf.nascar.com/cacher/2023/race_list_basic.json

class Schedule:
    
    def __init__(self, year, series_id):
        self.year = year
        self.series_id = series_id
        self.races = []
        self.data = pd.DataFrame()
        self.fetch_races()

    def fetch_races(self):
        """Fetch the race list for the specified year and series ID."""
        url = f"https://cf.nascar.com/cacher/{self.year}/race_list_basic.json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Filter the races by series ID
            race_list = data[f'series_{self.series_id}']
            self.races = [race for race in race_list if race['series_id'] == self.series_id]
            self.data = pd.DataFrame(self.races)
            
        else:
            warnings.warn(f"Failed to fetch race list: {response.status_code}")
    def get_completed_races(self):
        """Return a list of completed races."""
        return self.data[self.data['winner_driver_id'].notna()]['race_name'].tolist()


schedule = Schedule(year, series_id)
schedule.data.head()

,race_id,series_id,race_season,race_name,race_type_id,restrictor_plate,track_id,track_name,date_scheduled,race_date,...,satellite_radio_broadcaster,master_race_id,inspection_complete,playoff_round,is_qualifying_race,qualifying_race_no,qualifying_race_id,has_qualifying,winner_driver_id,pole_winner_laptime
0,5543,1,2025,Cook Out Clash at Bowman Gray,2,False,159,Bowman Gray Stadium,2025-02-02T20:00:00,2025-02-02T20:00:00,...,SIRIUSXM,3222,True,0,False,0,-1,False,4062.0,None
1,5544,1,2025,Duel 1 at DAYTONA,2,True,105,Daytona International Speedway,2025-02-13T19:00:00,2025-02-13T19:00:00,...,SIRIUSXM,3217,True,0,False,0,-1,False,4025.0,None
2,5545,1,2025,Duel 2 at DAYTONA,2,True,105,Daytona International Speedway,2025-02-13T20:45:00,2025-02-13T20:45:00,...,SIRIUSXM,3218,True,0,False,0,-1,False,4180.0,None
3,5546,1,2025,DAYTONA 500,1,True,105,Daytona International Speedway,2025-02-16T14:30:00,2025-02-16T14:30:00,...,SIRIUSXM,385,True,0,False,0,-1,False,4184.0,None
4,5547,1,2025,Ambetter Health 400,1,True,111,Atlanta Motor Speedway,2025-02-23T15:00:00,2025-02-23T15:00:00,...,SIRIUSXM,3335,True,0,False,0,-1,False,4153.0,None


In [38]:
#https://cf.nascar.com/cacher/2023/2/5314/lap-times.json
#

class Race:
    def __init__(self, year, series_id,race_id=None):
        self.year = year
        self.series_id = series_id
        self.race_id = race_id
        self.name = None
        self.laps = None
        self.winner = None
        self.data = pd.DataFrame()
        self.laps = pd.DataFrame()
        self.test = None
        
    def fetch_laps(self):
        """Fetch lap times for the specified race ID."""
        url = f"https://cf.nascar.com/cacher/{self.year}/{self.series_id}/{self.race_id}/lap-times.json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            self.data = data
            lap_times = []
            
            for i in data['laps']:
                driver = i['FullName']
                number = i['Number']
                manufacturer = i['Manufacturer']
                for j in i['Laps']:
                    lap_times = lap_times
                    lap_times.append({
                        'Driver': driver,
                        'Number': number,
                        'Manufacturer': manufacturer,
                        'Lap': j['Lap'],
                        'lap_time': j['LapTime'],
                        'lap_speed': j['LapSpeed'],
                        'position': j['RunningPos'],
                    })
            self.laps = pd.DataFrame(lap_times)
            self.laps['Lap'] = self.laps['Lap'].astype(int)
            self.laps['lap_time'] = pd.to_timedelta(self.laps['lap_time'])
            self.laps['lap_speed'] = self.laps['lap_speed'].astype(float)

race = Race(year, series_id, 5546)

race.fetch_laps()

In [36]:
laps = race.test['laps']
lap_times = []

for i in laps:
    driver = i['FullName']
    number = i['Number']
    manufacturer = i['Manufacturer']
    for j in i['Laps']:
        lap_times = lap_times
        lap_times.append({
            'Driver': driver,
            'Number': number,
            'Manufacturer': manufacturer,
            'Lap': j['Lap'],
            'lap_time': j['LapTime'],
            'lap_speed': j['LapSpeed'],
            'position': j['RunningPos'],
        })
        
lap_times = pd.DataFrame(lap_times)
lap_times

,Driver,Number,Manufacturer,Lap,lap_time,lap_speed,position
0,William Byron,24,Chv,0,NaN,None,5
1,William Byron,24,Chv,1,52.135,172.629,5
2,William Byron,24,Chv,2,47.861,188.045,5
3,William Byron,24,Chv,3,47.966,187.633,4
4,William Byron,24,Chv,4,47.957,187.668,2
...,...,...,...,...,...,...,...
7474,AJ Allmendinger,16,Chv,38,47.494,189.498,12
7475,AJ Allmendinger,16,Chv,39,47.584,189.139,15
7476,AJ Allmendinger,16,Chv,40,47.517,189.406,14
7477,AJ Allmendinger,16,Chv,41,47.506,189.450,14


In [40]:
race.laps.Driver.unique()

array(['William Byron', 'Tyler Reddick', '* Jimmie Johnson',
       'Chase Briscoe', 'John Hunter Nemechek', 'Alex Bowman',
       'Ryan Blaney', 'Austin Cindric', '* Justin Allgaier(i)',
       'Chris Buescher', 'Michael McDowell', 'Erik Jones',
       'Daniel Suarez', 'Ty Dillon', 'Chase Elliott', 'Ty Gibbs',
       'Riley Herbst #', 'Ricky Stenhouse Jr.', 'Justin Haley',
       'Kyle Larson', 'Cole Custer', '* Corey LaJoie', 'Austin Dillon',
       'Denny Hamlin', 'Cody Ware', 'Brad Keselowski', 'Todd Gilliland',
       'Noah Gragson', 'Bubba Wallace', 'Carson Hocevar',
       'Christopher Bell', 'Ryan Preece', 'Shane Van Gisbergen #',
       'Kyle Busch', 'Joey Logano', 'Zane Smith', 'Josh Berry',
       '* Martin Truex Jr.', '* Helio Castroneves', 'Ross Chastain',
       'AJ Allmendinger'], dtype=object)